In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
   # api_key=os.environ.get("OPENAI_API_KEY")
)

In [3]:
from autogen_core.models import UserMessage

result = await model_client.create(
    [UserMessage(content="Tell me about Elon Musk in 2 bullet points", source="user")]
    )
print(result)

finish_reason='stop' content='- **Entrepreneurial Ventures:** Elon Musk is known for founding and leading several high-profile companies, including Tesla, Inc., which revolutionized the electric vehicle market, and SpaceX, which aims to reduce space transportation costs and enable Mars colonization.\n\n- **Innovative Technologies:** Musk is a proponent of advanced technologies such as Neuralink, which focuses on brain-machine interfaces, and The Boring Company, which targets urban transportation challenges through tunneling solutions.' usage=RequestUsage(prompt_tokens=18, completion_tokens=93) cached=False logprobs=None thought=None


In [18]:
import os

def get_project_tree_string(project_path, max_depth=3, indent="    "):
    """
    Generates a tree-like string representation of a project directory up to a given depth.

    Args:
        project_path (str): The path to the project directory.
        max_depth (int): Maximum depth to traverse.
        indent (str): Indentation used for each level.

    Returns:
        str: A tree-like formatted string of the project structure.
    """
    def traverse(directory, depth=0, prefix=""):
        if depth > max_depth:
            return ""  # Stop recursion at max depth
        
        tree_str = ""
        try:
            items = sorted(os.listdir(directory))  # Sort to maintain order
            for i, item in enumerate(items):
                item_path = os.path.join(directory, item)
                is_last = (i == len(items) - 1)
                branch = "└── " if is_last else "├── "

                tree_str += f"{prefix}{branch}{item}\n"

                if os.path.isdir(item_path):
                    extension = "    " if is_last else "│   "
                    tree_str += traverse(item_path, depth + 1, prefix + extension)
        except PermissionError:
            tree_str += f"{prefix}└── [Permission Denied]\n"
        
        return tree_str

    return f"{os.path.basename(project_path)}\n" + traverse(project_path)

# Example usage
project_path = "C:\\Users\\vasanth.mu\\Desktop\\genai-training\\fin-5\\awesome-compose\\flask"
project_tree = get_project_tree_string(project_path)
print(project_tree)


flask
├── README.md
├── app
│   ├── Dockerfile
│   ├── app.py
│   └── requirements.txt
└── compose.yaml



In [29]:
#make the agent for docker file generation using the project strucure using python repl tool 
import pandas as pd
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import TextMessage
tool = LangChainToolAdapter(PythonAstREPLTool(locals={"project_path": project_path}))

# AI Model
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

# Define AI Assistant Agent
agent = AssistantAgent(
    "assistant",
    tools=[tool],
    model_client=model_client,
    system_message=(
        "You are an AI DevOps assistant. "
        "structure.use the python repl tool for getting the tree structure for the give project and then "
        "Your task is to generate a `Dockerfile` for the given project.. You can access the project path from the project_path variable"
        "\n\nProject path:\n" + project_path
    )
)

# Run AI Query
res = await Console(
    agent.on_messages_stream(
        [TextMessage(content="Generate a Dockerfile for this project.", source="user")], CancellationToken()
    ),
    
)

---------- assistant ----------
[FunctionCall(id='call_dXvf76DofxAy7bEtwqNjXptB', arguments='{"query":"import os\\nos.listdir(\'C:\\\\Users\\\\vasanth.mu\\\\Desktop\\\\genai-training\\\\fin-5\\\\awesome-compose\\\\flask\')"}', name='python_repl_ast')]
---------- assistant ----------
[FunctionExecutionResult(content="SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \\UXXXXXXXX escape (<unknown>, line 2)", name='python_repl_ast', call_id='call_dXvf76DofxAy7bEtwqNjXptB', is_error=False)]
---------- assistant ----------
SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<unknown>, line 2)


In [57]:
import os
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import TextMessage
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor

# Define project path
# Define project path
project_path = r"C:\Users\vasanth.mu\Desktop\genai-training\fin-5\awesome-compose\flask"

# Initialize Local Command Line Executor (Executes shell commands in the given directory)
local_executor = LangChainToolAdapter(LocalCommandLineCodeExecutor(work_dir=project_path))

agent = AssistantAgent(
    "assistant",
    tools=[local_executor],
    model_client=model_client,
    system_message=(
        "You are an AI DevOps assistant with expertise in containerization and project structuring. "
        "Follow these steps strictly:\n"
        "1. **Recursively list all files** in the project directory and its subdirectories.\n"
        "2. **Identify the tech stack** based on specific files:\n"
        "   - `requirements.txt` → Flask/Python (use `pip install` for dependencies)\n"
        "   - `package.json` → Node.js (use `npm install`)\n"
        "   - `pom.xml` → Java (use Maven `mvn install`)\n"
        "3. **Generate a fully optimized `Dockerfile`** based on the detected stack.\n"
        "4. **Ensure best practices**, including:\n"
        "   - Using lightweight base images (`python:3.9-slim`, `node:18-alpine`, etc.)\n"
        "   - Installing only necessary dependencies\n"
        "   - Setting appropriate working directories and entry points\n"
        "5. **Return the complete `Dockerfile` as the final response.**\n\n"
        "Project Structure Example:\n"
        "```\n"
        "flask/\n"
        "├── README.md\n"
        "├── app/\n"
        "│   ├── Dockerfile\n"
        "│   ├── app.py\n"
        "│   └── requirements.txt\n"
        "└── compose.yaml\n"
        "```\n\n"
        "Project Path:\n" + project_path
    ),
)

# Run AI Query
res = await Console(
    agent.on_messages_stream(
        [TextMessage(content="List all files and generate a Dockerfile based on the project structure.", source="user")],
        cancellation_token=CancellationToken(),
    )
)

print(res.chat_message.content)


C:\Users\vasanth.mu\AppData\Local\Temp\ipykernel_25904\577462038.py:15: UserWarning: The current event loop policy is not WindowsProactorEventLoopPolicy. This may cause issues with subprocesses. Try setting the event loop policy to WindowsProactorEventLoopPolicy. For example: `asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())`. See https://docs.python.org/3/library/asyncio-eventloop.html#asyncio.ProactorEventLoop.
  local_executor = LangChainToolAdapter(LocalCommandLineCodeExecutor(work_dir=project_path))


AttributeError: 'LocalCommandLineCodeExecutor' object has no attribute 'name'

In [58]:
import os
from typing import Dict, List, Optional, Union, Any
import json
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_core import CancellationToken
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import TextMessage
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
import asyncio

# Configure your model client (OpenAI, Azure, etc.)
from autogen_agentchat.llm_config import ChatCompletionConfig
model_config = ChatCompletionConfig(
    model="gpt-4",  # Replace with your model
    api_key=os.environ.get("OPENAI_API_KEY")  # Use your API key from environment variables
)

# Define project path - the path you want to containerize
project_path = r"C:\Users\vasanth.mu\Desktop\genai-training\fin-5\awesome-compose\flask"

# Create two local executors - one for examining files and one for Docker operations
file_explorer = LangChainToolAdapter(
    LocalCommandLineCodeExecutor(work_dir=project_path)
)

docker_executor = LangChainToolAdapter(
    LocalCommandLineCodeExecutor(work_dir=project_path)
)

# Create a Python REPL tool for more complex operations
python_repl = LangChainToolAdapter(PythonAstREPLTool())

# Define the agent workflow in separate functions for clarity
async def run_docker_containerization_workflow():
    """Main function to run the containerization workflow"""
    
    # Create the assistant agent with appropriate tools and system message
    assistant = AssistantAgent(
        "DockerBuilder",
        tools=[file_explorer, docker_executor, python_repl],
        model_client=model_config,
        system_message=(
            "You are an AI DevOps specialist with expertise in containerization. "
            "Follow these steps in order:\n\n"
            
            "1. **ANALYZE PROJECT STRUCTURE**:\n"
            "   - First, list all files in the project to understand its structure\n"
            "   - Use shell commands like `dir /s /b` (Windows) or `find . -type f | sort` (Linux/Mac)\n\n"
            
            "2. **IDENTIFY TECH STACK AND DEPENDENCIES**:\n"
            "   - Look for key files that identify the tech stack:\n"
            "     * `requirements.txt` → Python/Flask (examine contents for specific packages)\n"
            "     * `package.json` → Node.js (check dependencies section)\n"
            "     * `pom.xml` or `build.gradle` → Java\n"
            "     * `go.mod` → Go\n"
            "     * `composer.json` → PHP\n"
            "   - Check for framework-specific files: app.py (Flask), index.js (Node), etc.\n"
            "   - Examine configuration files (e.g., .env, config.yaml) for required environment variables\n\n"
            
            "3. **DETERMINE PORTS AND ENVIRONMENT REQUIREMENTS**:\n"
            "   - Look for port configurations in main application files\n"
            "   - Check for environment variables used by the application\n\n"
            
            "4. **CREATE OPTIMIZED DOCKERFILE**:\n"
            "   - Choose the most appropriate lightweight base image for the tech stack\n"
            "   - Include only necessary dependencies and build tools\n"
            "   - Follow Dockerfile best practices (multi-stage builds when appropriate)\n"
            "   - Set proper working directories, users, and permissions\n"
            "   - Configure appropriate EXPOSE directives for ports\n"
            "   - Add health checks if possible\n\n"
            
            "5. **VERIFY AND TEST**:\n"
            "   - Validate the Dockerfile syntax\n"
            "   - Suggest commands to build and test the Docker image\n\n"
            
            "Project Path: " + project_path
        ),
    )
    
    # Create a user proxy with appropriate configuration
    user = UserProxyAgent(
        "User",
        human_input_mode="NEVER",  # No human input needed for automation
        max_consecutive_auto_reply=10,
        default_auto_reply="Continue the analysis.",
    )
    
    # Run the conversation
    initial_message = TextMessage(
        content=(
            "Analyze the project at the specified path and create an optimized Dockerfile. "
            "Start by exploring the project structure, then identify the tech stack, and finally "
            "create a production-ready Dockerfile with best practices. Be systematic in your approach."
        ),
        source="User"
    )
    
    # Use Console to show the conversation stream
    result = await Console(
        assistant.on_messages_stream(
            [initial_message],
            cancellation_token=CancellationToken(),
        )
    )
    
    # Return the final result
    return result.chat_message.content

# Function to initialize and run the workflow
async def main():
    result = await run_docker_containerization_workflow()
    print("\n=== FINAL RESULT ===\n")
    print(result)

# Run the workflow
if __name__ == "__main__":
    asyncio.run(main())

ModuleNotFoundError: No module named 'autogen_agentchat.llm_config'

In [26]:
#make the agent for docker file generation using the project strucure using python repl tool 
import pandas as pd
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import TextMessage
tool = LangChainToolAdapter(PythonAstREPLTool(locals={"project_structure": project_tree}))

# AI Model
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

# Define AI Assistant Agent
agent = AssistantAgent(
    "assistant",
    tools=[tool],
    model_client=model_client,
    system_message=(
        "You are an AI DevOps assistant. Your task is to generate a `Dockerfile` for the given project "
        "structure."
        "\n\nProject Structure:\n" + project_tree
    )
)

# Run AI Query
res = await Console(
    agent.on_messages_stream(
        [TextMessage(content="Generate a Dockerfile for this project.", source="user")], CancellationToken()
    ),
    
)

---------- assistant ----------
[FunctionCall(id='call_539ArTuuJqoyY5v3b2EmOM2Y', arguments='{"query":"# Creating a Dockerfile for a Flask application\\n\\n# A Dockerfile typically contains the following instructions:\\n\\ndockerfile_content = \'\'\'\\n# Use the official Python base image\\nFROM python:3.9-slim\\n\\n# Set the working directory\\nWORKDIR /app\\n\\n# Copy the requirements file\\nCOPY app/requirements.txt .\\n\\n# Install dependencies\\nRUN pip install --no-cache-dir -r requirements.txt\\n\\n# Copy the rest of the application code\\nCOPY app/ .\\n\\n# Expose the application\'s port\\nEXPOSE 5000\\n\\n# Define the command to run the application\\nCMD [ \\"python\\", \\"app.py\\" ]\\n\'\'\' \\n\\ndockerfile_content"}', name='python_repl_ast')]
---------- assistant ----------
[FunctionExecutionResult(content='\n# Use the official Python base image\nFROM python:3.9-slim\n\n# Set the working directory\nWORKDIR /app\n\n# Copy the requirements file\nCOPY app/requirements.txt .\n

In [27]:
res.chat_message.content

'\n# Use the official Python base image\nFROM python:3.9-slim\n\n# Set the working directory\nWORKDIR /app\n\n# Copy the requirements file\nCOPY app/requirements.txt .\n\n# Install dependencies\nRUN pip install --no-cache-dir -r requirements.txt\n\n# Copy the rest of the application code\nCOPY app/ .\n\n# Expose the application\'s port\nEXPOSE 5000\n\n# Define the command to run the application\nCMD [ "python", "app.py" ]\n'

In [28]:
import yaml

def dockerfile_to_yaml(dockerfile_content):
    """
    Converts a Dockerfile string into a YAML-formatted multiline string.
    
    Args:
        dockerfile_content (str): The raw Dockerfile content.

    Returns:
        str: YAML string with Dockerfile content properly formatted.
    """
    yaml_data = {"dockerfile": dockerfile_content}  # Store Dockerfile under the "dockerfile" key
    return yaml.dump(yaml_data, default_style=None, default_flow_style=False)

# Example Dockerfile content (string)


# Convert to YAML
yaml_output = dockerfile_to_yaml(res.chat_message.content)

# Print YAML output
print(yaml_output)

# Optional: Save to a YAML file
with open("dockerfile.yaml", "w") as file:
    file.write(yaml_output)


dockerfile: '

  # Use the official Python base image

  FROM python:3.9-slim


  # Set the working directory

  WORKDIR /app


  # Copy the requirements file

  COPY app/requirements.txt .


  # Install dependencies

  RUN pip install --no-cache-dir -r requirements.txt


  # Copy the rest of the application code

  COPY app/ .


  # Expose the application''s port

  EXPOSE 5000


  # Define the command to run the application

  CMD [ "python", "app.py" ]

  '

